In [12]:
from TimeTagger import FileReader
import numpy as np
import easygui
import pandas as pd
import matplotlib.pyplot as plt



We know that H and V got swapped in BOB so I must correct for it in code. 

In [13]:
#GLOBAL DEFINITIONS
aH = 5
aV = 6
bH = 1
bV = 2
cH = 3
cV = 4

H=0
V=1

In [14]:
#folder = r'C:\Data\\'
#folder = "/media/tanvir/SpooqyLab/Table_Top_Demo/1stOct2020"
#folder = "./test"
#folder = "/home/tanvir/work/Programming/gProj/Satellite_QKD/data_source"
#folder = "/home/tanvir/work/Programming/gProj/qkd-table-top-v2/qutools"
#folder = "/home/tanvir/work/Programming/gProj/qkd-table-top-v2/test"
#folder = "/home/tanvir/work/Programming/gProj/qkd-table-top-v2/swabian/newmatch11Jun21"
folder = "./VerifiableRNG_Data/logSwabian20220221-1433/"
#file = '2.97mW_1.1.ttbin'
#file = "onlyB.1.ttbin"
#file = "dark1.ttbin"
#file = "3_88mW00dB.1.ttbin"
file ="logSwabian20220221-1433.ttbin"
filepath = folder + '/' + file

#filepath = easygui.fileopenbox(default = folder)






In [15]:
def loadSwabianTimeStamps(fileIn, t = np.inf):
    
        file_reader = FileReader(fileIn)
        stamps = []
        channel = []
        
        
        while file_reader.hasData():
            #n_events = 100000; # Number of events to read at once
            n_events = 400000; # Number of events to read at once
            
            # Read at most n_events.
            data = file_reader.getData(n_events);  # data is an instance of TimeTagStreamBuffer

            channelTmp = data.getChannels()
            stampsTmp = data.getTimestamps()
          #  overflow_types = data.getEventTypes()   # TimeTag = 0, Error = 1, OverflowBegin = 2, OverflowEnd = 3, MissedEvents = 4
          #  missed_events = data.getMissedEvents()
            
            for i in range(len(channelTmp)):
                channel.append(channelTmp[i])
                stamps.append(stampsTmp[i])
            break
            if stamps[-1] - stamps[0] > t:
                break
        return stamps, channel


In [16]:
stamps, channel = loadSwabianTimeStamps(filepath)

In [17]:
#for (x,y) in zip(stamps,channel):
#    print (x,y)

In [18]:
#seperate out alice's, bob's and charlie's events
astamps = []
achannel = []
bstamps = []
bchannel = []
cstamps = []
cchannel = []

for (s,c) in zip(stamps, channel):
    if (c==aH or c==aV):
        astamps.append(s+2000)
        achannel.append(c-aH)
    elif (c==bH or c==bV):
        bstamps.append(s)
        bchannel.append(c-bH)
    elif (c==cH or c==cV):
        cstamps.append(s)
        cchannel.append(c-cH)
    else :
        print ("Error! Unknown channel: ",c)


In [19]:
#len(astamps)+len(bstamps)+len(cstamps)
len(achannel)+len(bchannel)+len(cchannel)

400000

In [20]:
detectors = ["H","V"]

In [21]:
def correct_timestamp(stamps,detectors,step_size_ps=250,sH=0,sV=0):
    cstamp=[]
    ts = 0
    for (s,d) in zip (stamps,detectors):
        ts = s
        if(d==H): 
            ts+=sH*step_size_ps
        elif(d==V):
            ts+=sV*step_size_ps 
        cstamp.append(ts)
        
    print (step_size_ps*sH,step_size_ps*sV)
    
    return cstamp

In [22]:
#correct alice HV delay missmatch add 500ps to alice's V 
#astamps_corrected = correct_timestamp(astamps,achannel,sH=0,sV=-1*500,sA=0,sD=-2*500) #28 july data
#astamps_corrected = correct_timestamp(astamps,achannel,sH=0,sV=-2*500,sA=0,sD=-2*500) # 24 Aug data
#astamps_corrected = correct_timestamp(astamps,achannel,sH=0,sV=-2*500,sA=-3*500,sD=-1*500) # 23rd Oct data
#astamps_corrected = correct_timestamp(astamps,achannel,step_size_ps=250,sH=0,sV=-3,sA=-5,sD=-1) # 26rd Oct data
#astamps_corrected = correct_timestamp(astamps,achannel,step_size_ps=250,sH=0,sV=-3,sA=-6,sD=0) #23rd Oct new calibration
bstamps_corrected = correct_timestamp(bstamps,bchannel,step_size_ps=250,sH=0,sV=2)

0 500


In [23]:
#correct charlie
cstamps_corrected = correct_timestamp(cstamps,cchannel,step_size_ps=250,sH=-1,sV=2)

-250 500


In [24]:
hh = {}

In [25]:
def match_coincidence(at,ad,bt,bd,cwindow,shift):
    ia = 0
    ib = 0
    acoin_indexes = []
    bcoin_indexes = []
    while (ia<len(at) and ib <len(bt)):
        
        diff = bt[ib]- at[ia] + shift

        if (abs(diff)<=cwindow):
            #coin_count +=1
            if ad[ia]==bd[ib]:
                acoin_indexes.append(ia)
                bcoin_indexes.append(ib)
            ia+=1
            ib+=1
            #continue
        elif (diff>cwindow):
            ia+=1
            #continue
        elif( diff <-cwindow):
            ib+=1
            #continue
        else:
            print ("ERROR! I should not be here")
            break
    return acoin_indexes,bcoin_indexes



In [26]:
ab_indexes,bc_indexes = match_coincidence(astamps,achannel,bstamps_corrected,bchannel,750,4*250)

In [27]:
len(ab_indexes)



2546

In [28]:
1256 + 592

1848

In [29]:
ac_indexes,cc_indexes = match_coincidence(astamps,achannel,cstamps_corrected,cchannel,750,4*250)

In [30]:
def generate_tripartite_output(ab_indexes,ac_indexes,astamps,achannel):
    columnA=[]
    columnB=[]
    columnC=[]
    ib=0
    ic=0
    tag_ib=None
    tag_ic=None
    a=0
    b=0
    c=0
    while(ib < len(ab_indexes) and ic <len(ac_indexes)):
        tag_ib = ab_indexes[ib]
        tag_ic = ac_indexes[ic]
        
        if tag_ib < tag_ic: 
            #process tag_ib
            bit= achannel[tag_ib]
            a = 0
            b = bit
            c = bit
            ib+=1
        elif tag_ib > tag_ic:
            #process gat_ic
            bit = achannel[tag_ic]
            a = 1
            b = bit
            c = 1-bit
            ic +=1
        else:
            ib+=1
            ic+=1
        columnA.append(a)
        columnB.append(b)
        columnC.append(c)
    
    return columnA,columnB,columnC
    

In [31]:
A,B,C=generate_tripartite_output(ab_indexes,ac_indexes,astamps,achannel)


In [32]:
A[:20]

[1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1]

In [33]:
B[:20]

[1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0]

In [34]:
C[:20]

[0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1]

In [35]:
df = pd.DataFrame()
df['columnA'] = A
df['columnB'] = B
df['columnC'] = C
df.to_csv("PQRNG_output.csv",index=False)


In [38]:
print(sum(A),sum(B),sum(C),len(A))

3000 2735 2807 5546


In [37]:
def hex_from_bitmap(ap,bp):
    h=0
    for i in ap: 
        h<<=1
        h= h|i
    for i in bp: 
        h<<=1
        h= h|i    
    #print (hex(h) )
    return h
    
hex_from_bitmap([0,0,1,0],[0,0,1,0])

34